In [ ]:
!pip install ccxt

In [ ]:
import ccxt

## Obtain exchanges

In [ ]:
exchanges = ccxt.exchanges
len(exchanges)

108

In [ ]:
exchanges[:5]

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone']

## Connect to an exchange

In [ ]:
binance = ccxt.binance()

## Obtain data

In [ ]:
kraken = ccxt.kraken()

In [ ]:
#btc_binance = binance.fetch_ohlcv('BTC/USDT','1d',limit=50)
btc_kraken = kraken.fetch_ohlcv('BTC/USDT','1d',limit=50)

In [ ]:
btc_kraken[:3]

[[1621036800000, 49832.7, 50640.3, 46608.9, 46705.9, 437.73057172],
 [1621123200000, 46751.3, 49760.0, 43850.2, 46394.1, 763.51353804],
 [1621209600000, 46417.5, 46606.2, 42100.0, 43552.3, 905.7982499]]

## Pass API Keys

In [ ]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)
binance = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})

## Market Order

In [ ]:
order = kraken.create_order(
    symbol="ETH/BTC",
    type="market",
    side="sell",
    amount=10
)

## Limit Order

In [ ]:
symbol = 'ETH/BTC'
type = 'limit'
side = 'sell'
amount = 123.45
price = 54.321
params = {
    'triggerPrice': 123.45,
}

order = kraken.create_order(symbol, type, side, amount, price, params)

## Set Sandbox

In [ ]:
binance = ccxt.binance()
binance.set_sandbox_mode(True)

## Trade Example

In [ ]:
from time import sleep

In [ ]:
exchange_id = 'kraken'
exchange_class = getattr(ccxt, exchange_id)
kraken = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})

In [ ]:
while True:
    try:
        orderbook = binance.fetch_order_book("BTC/USDT")
    except Exception as e:
        print(f"Unable to fetch orderbook from Binance: {e}")
    ask_old = orderbook["asks"][0][0] if len(orderbook["asks"]) > 0 else None

    sleep(300)

    try:
        orderbook = binance.fetch_order_book("BTC/USDT")
    except Exception as e:
        print(f"Unable to fetch orderbook from Binance: {e}")
    ask_new = orderbook["asks"][0][0] if len(orderbook["asks"]) > 0 else None

    percentage_change = ((ask_new - ask_old) / ask_old) * 100
    print(f"BTC has moved {percentage_change} percent in the last 5 minutes")

    if percentage_change >= 5:
        print("Buying BTC on Kraken")
        kraken_orderbook = kraken.fetch_order_book("BTC/USDT")
        kraken_ask = (
            kraken_orderbook["asks"][0][0]
            if len(kraken_orderbook["asks"]) > 0
            else None
        )
        try:
            order = kraken.create_order(
                symbol="BTC/USDT",
                type="limit",
                side="buy",
                amount=20,
                price=kraken_ask + 1,
            )
        except Exception as e:
            print(f"Unable to place order on Kraken: {e}")
            break

        print(order)
        break

## Arbitrage Screener Example

In [ ]:
exchanges = ["binance", "kraken", "bitfinex"]
symbols = ["BTC/USDT", "ETH/USDT", "ETH/BTC"]

for exchange_id in exchanges:
    exchange = getattr(ccxt, exchange_id)()
    markets = exchange.load_markets()
    for symbol in symbols:
        if symbol not in exchange.symbols:
            print(symbol, "not found in", exchange_id)

In [ ]:
from time import sleep

In [ ]:
def get_price(exchange_id, symbol):
    exchange = getattr(ccxt, exchange_id)()
    exchange.load_markets()
    orderbook = exchange.fetch_order_book(symbol)
    bid = orderbook["bids"][0][0] if len(orderbook["bids"]) > 0 else None
    ask = orderbook["asks"][0][0] if len(orderbook["asks"]) > 0 else None
    return bid, ask


def check_price(exchange_id_1, exchange_id_2, symbol):
    bid1, ask1 = get_price(exchange_id_1, symbol)
    bid2, ask2 = get_price(exchange_id_2, symbol)
    bid_diff = bid1 - bid2
    ask_diff = ask1 - ask2

    if bid_diff > 0.2 or ask_diff > 0.2:
        print(
            f"=============================\n"
            f"ALERT {exchange_id_1} <> {exchange_id_2} | {symbol}\n"
            f"{exchange_id_1} bid: {bid1} | {exchange_id_2} bid: {bid2} | diff: {bid_diff}\n"
            f"{exchange_id_1} ask: {ask1} | {exchange_id_2} ask: {ask2} | diff: {ask_diff}\n"
            f"=============================\n"
        )
    else:
        print(
            f"{exchange_id_1} <> {exchange_id_2} | {symbol} bid_diff: {bid_diff} - ask_diff: {ask_diff}"
        )


while True:
    for exchange_id_1 in exchanges:
        for exchange_id_2 in exchanges:
            if exchange_id_1 != exchange_id_2:
                for symbol in symbols:
                    check_price(exchange_id_1, exchange_id_2, symbol)
    sleep(5)